# Arbeitspaket (AP) 3: Management & Nutzung Räumliche Daten

### Angaben Studierende(r) (fehlende Angaben ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td>Phil</td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td>Frei</td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td>21-548-946</td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>06.10.2025 / Raum: MU O2.001 / 8:00 – 11:45</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.25HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

### <b style="color: gray;">(max. erreichbare Punkte: 40)</b>

<table>
  <thead>
    <tr>
      <th>Kategorie</th>
      <th>Beschreibung</th>
      <th>Punkteverteilung</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Code nicht lauffähig oder Ergebnisse nicht sinnvoll</td>
      <td>Der Code enthält Fehler, die verhindern, dass er ausgeführt werden kann (z.B. Syntaxfehler) oder es werden Ergebnisse ausgegeben, welche nicht zur Fragestellung passen.</td>
      <td>0 Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit gravierenden Mängeln</td>
      <td>Der Code läuft, aber die Ergebnisse sind aufgrund wesentlicher Fehler unvollständig (z.B. fehlende Joins, gravierende Fehler in SQL-Abfragen). Nur geringer Fortschritt erkennbar.</td>
      <td>25% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit mittleren Mängeln</td>
      <td>Der Code läuft und liefert teilweise korrekte Ergebnisse, aber es gibt grössere Fehler (z.B. fehlende Spalten, unvollständige SQL-Abfragen). Die Ergebnisse sind nachvollziehbar, aber unvollständig oder ungenau.</td>
      <td>50% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit minimalen Mängeln</td>
      <td>Der Code läuft und liefert ein weitgehend korrektes Ergebnis, aber kleinere Fehler (z.B. falsche oder fehlende Sortierung, Rundung von Werten falsch) beeinträchtigen die Vollständigkeit des Ergebnisses.</td>
      <td>75% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig und korrekt</td>
      <td>Der Code läuft einwandfrei und liefert das korrekte Ergebnis ohne Mängel.</td>
      <td>100% der max. erreichbaren Punkte</td>
    </tr>
  </tbody>
</table>


#### <b>Python Libraries und Settings</b>

In [80]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from IPython.display import display, IFrame
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())


/workspaces/python_postgresql_postgis


## <b>Vorbereitung (Hinweis: dieser Teil wird <u>nicht</u> bewertet)</b>

#### <b>1.) Starten Sie eine GitHub Codespaces Instanz auf Basis Ihres Forks des folgenden GitHub Repositories:</b>

##### GitHub-Repository: https://github.com/mario-gellrich-zhaw/python_postgresql_postgis

##### <span style="color: red;"><b>WICHTIG!!! Verwenden Sie eine GitHub Codespaces Instanz mit ausreichend Arbeitsspeicher (4core, 16GB RAM).</b></span>

<b>Hinweis:</b> 
- Im Unterricht wurden bereits sämtliche Installationen und Einstellungen inkl. der Registrierung des Datenbank Servers auf pgAdmin vorgenommen.
- Falls Sie die Codespaces-Instanz neu erstellen müssen, folgen Sie bitte den detaillierten Erklärungen auf der README-Seite des GitHub Repositories.

#### <b>2.) Erstellen und Testen Sie die Datenbankverbindung mit der 'osm_switzerland' Datenbank.</b>

In [81]:
# Set up Database Connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Erstellen der Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Erstellen SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test der Connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT current_database();"))
        db_name = result.scalar()
        print("Aktuelle Datenbank:", db_name)
except Exception as e:
    print("Fehler bei der Verbindung:", e)

# Verbindung trennen
engine.dispose()

Aktuelle Datenbank: osm_switzerland


#### <b>3.) Legen Sie einen neuen Order 'maps' im Arbeitsverzeichnis an. In diesem werden Ihre generierten Karten abgespeichert.</b>

<b style="color:red;">Hinweis: Der Order 'maps' inklusive die erzeugten Karten muss als Teil des AP03 mit auf EXAM-Moodle abgegeben werden!</b>

In [82]:
# Anlegen eines Ordners 'maps'
if not os.path.exists('maps'):
    os.makedirs('maps')
    print("Ordner 'maps' wurde erstellt.")
else:
    print("Ordner 'maps' existiert bereits und wird nicht neu angelegt!")

Ordner 'maps' existiert bereits und wird nicht neu angelegt!


## <b>Aufgaben (Dieser Teil wird bewertet!)</b>
<b>Hinweise zu den folgenden Aufgabenstellungen:</b>
<ul>
  <li>In diesem Jupyter Notebook gibt es jeweils zwei Code-Zellen pro Aufgabe:</li>
  <ol>
    <li>Eine Codezelle mit Python-Code für die SQL-Abfrage.</li>
    <li>Eine Codezelle mit Python-Code für die Kartendarstellung der Ergebnisse der jeweiligen SQL-Abfrage.</li>
  </ol>
  <li>In den Codezellen für die SQL-Abfrage muss jeweils die SQL-Statement ergänzt werden.</li>
  <li>In den Codezellen für die Kartendarstellung muss nur dann der Python Code ergänzt werden, wenn in der Aufgabe danach gefragt wird.</li>
</ul>
<b style="color:red;">Beachten Sie, dass für die Punktevergabe auch die weiteren Anforderungen zu den Fragen unter 'Details zur Aufgabenstellung' herangezogen werden.</b>

#### <b>Aufgabe (1): Erstellen Sie eine Abfrage der Spitäler.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Daten in der Tabelle 'planet_osm_polygon'.
- Stellen sie in der Ergebnistabelle die Spalten: osm_id, amenity sowie die transformierte Geometrie als Spalte geom dar.
- Tipps:
    - Die Geometry wird mit Hilfe der Funktion st_transform() transformiert, z.B.: *st_transform(p.way, 4326) AS geom*.
    - Die Daten sind mit dem key:value Paar amenity='hospital' in der OpenStreetMap Map-Feature Übersicht angegeben.
    - vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [83]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.amenity,
            ST_Transform(p.way, 4326) AS geom
         FROM
            planet_osm_polygon AS p
         WHERE
            p.amenity = 'hospital';"""

# Ergebnis in GeoDataFrame abspeichern
gdf_hospital = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf_hospital.head(10)

,osm_id,amenity,geom
0,165785776,hospital,"POLYGON ((8.7261 47.50726, 8.7262 47.50723, 8...."
1,856698475,hospital,"POLYGON ((8.56529 47.44926, 8.56576 47.44914, ..."
2,612288154,hospital,"POLYGON ((8.54828 47.38012, 8.5483 47.38012, 8..."
3,-18241258,hospital,"POLYGON ((8.57208 47.35379, 8.57228 47.35284, ..."
4,-18241258,hospital,"POLYGON ((8.56919 47.3518, 8.56975 47.35114, 8..."
5,675191947,hospital,"POLYGON ((8.59648 47.34771, 8.5972 47.34769, 8..."
6,28530564,hospital,"POLYGON ((8.79935 47.32184, 8.79939 47.32181, ..."
7,-17941802,hospital,"POLYGON ((8.71801 47.26955, 8.71837 47.26926, ..."
8,317763534,hospital,"POLYGON ((8.58826 47.26274, 8.58826 47.26272, ..."
9,-9065183,hospital,"POLYGON ((8.45701 47.28345, 8.4573 47.28318, 8..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [84]:
# Projektion definieren (WGS84)
if gdf_hospital.crs is None:
    gdf_hospital.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_hospital.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='CartoDBPositron')

# Map settings
folium.GeoJson(
    gdf_hospital,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_hospital.html')
print("Die Karte wurde unter 'maps/map_hospital.html' gespeichert.")

Die Karte wurde unter 'maps/map_hospital.html' gespeichert.


#### <b>Aufgabe (2) Erstellen Sie eine Abfrage der SBB Bahnhöfe.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_point'.
- Schränken Sie nach Bahnhöfen der SBB ein.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, name, operator, public_transport und die transformierte Geometrie als Spalte geom dar.
- Tipp: Die Daten sind mit dem key:value Paar public_transport='station' in der OpenStreetMap Map-Feature Übersicht angegeben.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [85]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            h.osm_id,
            h.operator,
            h.public_transport,
            ST_Transform(h.way, 4326) AS geom

         FROM
            planet_osm_point AS h
         WHERE
            h.public_transport = 'station'
            AND h.operator = 'SBB';"""

# Ergebnis in GeoDataFrame abspeichern
gdf_stations = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf_stations.head(10)

,osm_id,operator,public_transport,geom
0,4574932116,SBB,station,POINT (8.85436 47.26035)
1,2414238691,SBB,station,POINT (8.85245 47.26845)
2,2443333883,SBB,station,POINT (8.93238 47.35388)
3,2141115746,SBB,station,POINT (8.92329 47.33342)
4,2443368760,SBB,station,POINT (8.85423 47.39426)
5,307391788,SBB,station,POINT (8.86333 47.49848)
6,1352757256,SBB,station,POINT (8.77602 47.52553)
7,1311007154,SBB,station,POINT (8.76012 47.50767)
8,1351838940,SBB,station,POINT (8.76892 47.5015)
9,2446019572,SBB,station,POINT (8.81905 47.44154)


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [86]:
# Projektion definieren (WGS84)
if gdf_stations.crs is None:
    gdf_stations.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_stations.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=10, 
               tiles='CartoDBPositron')

# Map settings with popup
folium.GeoJson(
    gdf_stations,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_stations.html')
print("Die Karte wurde unter 'maps/map_stations.html' gespeichert.")

Die Karte wurde unter 'maps/map_stations.html' gespeichert.


#### <b>Aufgabe (3): Erstellen Sie eine Abfrage aller Strassen, die als 'primary', 'secondary' oder 'tertiary' klassifiziert sind mit Infos zur Art des Fahrradwegs.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'planet_osm_line'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, highway, bicycle und die transformierte Geometrie als Spalte geom dar.
- Schränken Sie die Abfrage auf alle Strassen ein, bei denen Information zur Art des Fahrradweges vorliegen.
- Tipp: Sie finden die Informationen zur Art der Fahrradwegs in der Spalte 'bicycle'.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [87]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            h.osm_id,
            h.highway,
            h.bicycle,
            ST_Transform(h.way, 4326) AS geom
         FROM
            planet_osm_line AS h
         WHERE
            h.bicycle IS NOT NULL
            AND h.highway IN ('tertiary','primary','secondary');"""

# Ergebnis in GeoDataFrame abspeichern
gdf_streets = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf_streets.head(10)

,osm_id,highway,bicycle,geom
0,315007668,primary,use_sidepath,"LINESTRING (8.83126 47.24526, 8.83146 47.24537..."
1,1188796458,primary,use_sidepath,"LINESTRING (8.83238 47.24567, 8.83257 47.24569..."
2,1188796459,primary,use_sidepath,"LINESTRING (8.83489 47.24705, 8.83486 47.24703)"
3,315007660,primary,use_sidepath,"LINESTRING (8.83486 47.24693, 8.83496 47.247)"
4,31726656,primary,use_sidepath,"LINESTRING (8.83496 47.247, 8.83522 47.24717, ..."
5,1188796454,primary,use_sidepath,"LINESTRING (8.83529 47.24731, 8.83507 47.24717..."
6,265787852,primary,use_sidepath,"LINESTRING (8.83433 47.24651, 8.83447 47.24663..."
7,1188796450,primary,use_sidepath,"LINESTRING (8.83486 47.24703, 8.83464 47.24688..."
8,315007661,primary,use_sidepath,"LINESTRING (8.83361 47.24607, 8.83388 47.24621..."
9,315007654,primary,use_sidepath,"LINESTRING (8.83904 47.24789, 8.83933 47.24794..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [88]:
# Projektion definieren (WGS84)
if gdf_streets.crs is None:
    gdf_streets.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_streets.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='CartoDBPositron')

# Map settings
folium.Choropleth(
    geo_data=gdf_streets,
    name='map',
    line_weight=3,
    line_color='darkred'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_streets.html')
print("Die Karte wurde unter 'maps/map_streets.html' gespeichert.")

Die Karte wurde unter 'maps/map_streets.html' gespeichert.


#### <b>Aufgabe (4): Erstellen Sie eine Abfrage aller Gebäude in der Bahnhofstrasse in Zürich, bei denen mindestens die Postleitzahl (PLZ) vorhanden ist.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_polygon'.
- Stellen Sie in der Ergebnistabelle Strassenname, Haunummer, PLZ, Gemeindename sowie die transformierte Geometrie als Spalte geom dar.
- Dabei ist nicht wichtig, dass alle Adress-Attribute für jedes Gebäude vorhanden sind, nur die PLZ muss immer vorhanden sein.
- Verwenden Sie für die Darstellung als Hintergrundkarte ein Satellitenbild (ESRIWorldImagery) als maptile.
- Tipp: Gebäude sind in der Spalte 'building' klassifiziert. Mit *WHERE building IS NOT NULL* können Sie Gebäude filtern.

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [89]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            h."addr:street",
            h."addr:housenumber",
            h."addr:postcode",
            h."addr:city",
            ST_Transform(h.way, 4326) AS geom
         FROM
            planet_osm_polygon as h
         WHERE
            h."addr:postcode" IS NOT NULL
            AND h."addr:street" = 'Bahnhofstrasse'
            AND h."addr:city" = 'Zürich';"""

# Ergebnis in GeoDataFrame abspeichern
gdf_buildings = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf_buildings.head(10)

,addr:street,addr:housenumber,addr:postcode,addr:city,geom
0,Bahnhofstrasse,108,8001,Zürich,"POLYGON ((8.53988 47.37663, 8.54007 47.37658, ..."
1,Bahnhofstrasse,106,8001,Zürich,"POLYGON ((8.53983 47.37652, 8.53997 47.37648, ..."
2,Bahnhofstrasse,78,8001,Zürich,"POLYGON ((8.53898 47.37488, 8.53917 47.37483, ..."
3,Bahnhofstrasse,80,8001,Zürich,"POLYGON ((8.53903 47.37497, 8.53922 47.37492, ..."
4,Bahnhofstrasse,76,8001,Zürich,"POLYGON ((8.53893 47.37477, 8.53913 47.37473, ..."
5,Bahnhofstrasse,94,8001,Zürich,"POLYGON ((8.53952 47.37592, 8.53965 47.37589, ..."
6,Bahnhofstrasse,98,8001,Zürich,"POLYGON ((8.53959 47.37605, 8.53978 47.37601, ..."
7,Bahnhofstrasse,100,8001,Zürich,"POLYGON ((8.53964 47.37614, 8.53975 47.37612, ..."
8,Bahnhofstrasse,83,8001,Zürich,"POLYGON ((8.53901 47.37611, 8.53906 47.37609, ..."
9,Bahnhofstrasse,77,8001,Zürich,"POLYGON ((8.53838 47.37504, 8.5387 47.37496, 8..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [90]:
# Projektion definieren (WGS84)
if gdf_buildings.crs is None:
    gdf_buildings.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_buildings.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='EsriWorldImagery')

# Map settings
folium.Choropleth(
    geo_data=gdf_buildings,
    name='map',
    fill_color='orange'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_buildings.html')
print("Die Karte wurde unter 'maps/map_buildings.html' gespeichert.")

Die Karte wurde unter 'maps/map_buildings.html' gespeichert.


#### <b>Aufgabe (5): Erstellen Sie eine Abfrage des Greifensees, und generieren Sie einen Buffer um diesen See mit einer Breite von 500m.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'planet_osm_polygon'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, name, natural sowie die transformierte Geometrie des Buffers als Spalte geom dar.
- Im Polygondatensatz gibt es unter dem Namen 'Greifensee' nicht nur einen Polygon. Filtern Sie in der Abfrage nur den See.
- Tipp: Zum Filtern des Sees können Sie die Spalte 'natural' verwenden.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [91]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            h.osm_id,
            h.name,
            h.natural,
            ST_Buffer(ST_Transform(h.way::geometry, 3857), 500), 4326 AS geom         
        FROM
            planet_osm_polygon AS h
         WHERE
            h.name = 'Greifensee'
            AND h.natural = 'water';"""

# Ergebnis in GeoDataFrame abspeichern
gdf_lake = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()


# Zeigen des GeoDataFrames
gdf_lake





GEOSException: ParseException: Unexpected EOF parsing WKB

##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [ ]:
# Projektion definieren (WGS84)
if gdf_lake.crs is None:
    gdf_lake.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_lake.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='ESRIWorldImagery')

# Map settings
folium.Choropleth(
    geo_data=gdf_lake,
    name='map',
    fill_color='greenyellow',
    fill_opacity=0.5,
    line_opacity=0.5
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_lake.html')
print("Die Karte wurde unter 'maps/map_lake.html' gespeichert.")

ValueError: Location values cannot contain NaNs.

#### <b>Aufgabe (6): Erstellen Sie eine Abfrage der landwirtschaftlichen Nutzflächen in den Grenzen der Stadt Zürich.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Daten in der Tabelle 'planet_osm_polygon'.
- Landwirtschaftliche Nutzflächen sollen in dieser Aufgabe sowohl 'farmland' als auch 'meadows' beinhalten.
- Selektieren Sie die landwirtschaftlichen Nutzflächen über die Gemeindefläche.
- Berechen Sie die Fläche der landwirtschaftlichen Nutzflächen und speichern diese in einer Spalte 'area_ha' ab.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, landuse, area_ha sowie die transformierte Geometrie als geom dar.
- Wählen Sie eine Satelliten Karte von ESRI als Hintergrundkarte (maptile).
- Ordnen Sie die Ergebnistabelle nach area_ha absteigend.
- Tipp: Im Unterricht wurde eine ähnliche Abfrage mit Waldflächen (forest) behandelt.
- vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [103]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)
# SQL Beispiele mit PG Admin 2 Ordner (SQL Advanced)
# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            p.osm_id,
            p.landuse,
            ST_Area(
                ST_Transform(
                  ST_Intersection(p.way, z.way),
                  32632
            )
    ) / 10000 AS area_ha,
    ST_Transform(
        ST_Intersection(p.way, z.way),
        4326
         ) AS clipped_geom
      FROM planet_osm_polygon AS p
      JOIN planet_osm_polygon AS z
         ON ST_Intersects(z.way, p.way)
      WHERE
         z.admin_level = '8'
         AND z.name = 'Zürich'
         AND p.landuse IN ('farmland', 'meadows')
      ORDER BY 
         area_ha DESC;"""

# Ergebnis in GeoDataFrame abspeichern
gdf_agriculture = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()


# Zeigen des GeoDataFrames
gdf_agriculture.head(10)

ValueError: Query missing geometry column 'geom'

In [ ]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            h.osm_id,
            h.highway,
            h.bicycle,
            ST_Transform(h.way, 4326) AS geom
         FROM
            planet_osm_line AS h
         WHERE
            h.bicycle IS NOT NULL
            AND h.highway IN ('tertiary','primary','secondary');"""

# Ergebnis in GeoDataFrame abspeichern
gdf_streets = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf_streets.head(10)

,osm_id,highway,bicycle,geom
0,315007668,primary,use_sidepath,"LINESTRING (8.83126 47.24526, 8.83146 47.24537..."
1,1188796458,primary,use_sidepath,"LINESTRING (8.83238 47.24567, 8.83257 47.24569..."
2,1188796459,primary,use_sidepath,"LINESTRING (8.83489 47.24705, 8.83486 47.24703)"
3,315007660,primary,use_sidepath,"LINESTRING (8.83486 47.24693, 8.83496 47.247)"
4,31726656,primary,use_sidepath,"LINESTRING (8.83496 47.247, 8.83522 47.24717, ..."
5,1188796454,primary,use_sidepath,"LINESTRING (8.83529 47.24731, 8.83507 47.24717..."
6,265787852,primary,use_sidepath,"LINESTRING (8.83433 47.24651, 8.83447 47.24663..."
7,1188796450,primary,use_sidepath,"LINESTRING (8.83486 47.24703, 8.83464 47.24688..."
8,315007661,primary,use_sidepath,"LINESTRING (8.83361 47.24607, 8.83388 47.24621..."
9,315007654,primary,use_sidepath,"LINESTRING (8.83904 47.24789, 8.83933 47.24794..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [ ]:
# Projektion definieren (WGS84)
if gdf_agriculture.crs is None:
    gdf_agriculture.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_agriculture.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf_agriculture,
    name='map',
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_farmland_meadow.html')
print("Die Karte wurde unter 'maps/map_farmland_meadow.html' gespeichert.")

#### <b>Aufgabe (7): Erstellen Sie eine Abfrage der Hotels in einem Radius von 5km um den Flughafen Zürich.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_point'.
- Berechnen Sie in der Abfrage die Distanz jedes Hotels zum Flughafen in Metern als Spalte 'distance_meters'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, tourism, name, distance_meters, sowie die transformierte Geometrie als geom dar.
- Wählen Sie eine Satelliten Karte von ESRI als Hintergrundkarte (maptile).
- Integrieren Sie in die Kartendarstellung den Namen (Spalte 'name') der Hotels als Popup.
- Tipp: Hotels sind mit dem key:value Paar tourism='hotel' in der OpenStreetMap Map-Feature Übersicht angegeben.
- vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [ ]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
            h.osm_id,
            h.tourism,
            h.name,
            ST_Distance(
                ST_Transform(h.way, 4326)::geography,
                ST_SetSRID(ST_MakePoint(8.55337, 47.46339), 4326)::geography
            ) AS distance_meters,
            ST_TRANSFORM(h.way, 4326) AS geom
         FROM
            planet_osm_point AS h
         WHERE
         h.tourism = 'hotel'
         AND ST_DWithin(
            ST_Transform(h.way, 4326)::geography,
            ST_SetSRID(ST_MakePoint(8.55337, 47.46339), 4326)::geography, 5000
 );"""

# Ergebnis in GeoDataFrame abspeichern
gdf_hotels = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_hotels.head(10)


,osm_id,tourism,name,distance_meters,geom
0,6286669885,hotel,Welcome Inn,2653.371809,POINT (8.5816 47.44914)
1,12072254954,hotel,Sleepz,2704.658411,POINT (8.58278 47.44947)
2,6286669886,hotel,Fly away,2759.969866,POINT (8.58426 47.45007)
3,11274643146,hotel,Tailormade BAB,4703.356024,POINT (8.54166 47.50494)
4,5309146824,hotel,Gasthof Hecht,3197.908202,POINT (8.54776 47.4919)
5,8607233055,hotel,Transit Hotel & Dayrooms,1515.933550,POINT (8.56018 47.45056)
6,3086649239,hotel,Dorint Airport-Hotel Zürich,2893.652484,POINT (8.56559 47.43872)
7,3783549518,hotel,STAY@Zurich Airport,2911.537464,POINT (8.56747 47.43901)
8,4269704591,hotel,Apart-Hotel Zürich by Hilton,3183.671159,POINT (8.57373 47.43831)
9,2030250479,hotel,Welcome Homes,3694.085360,POINT (8.56689 47.43146)


In [ ]:
# Projektion definieren (WGS84)
if gdf_hotels.crs is None:
    gdf_hotels.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_hotels.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf_hotels,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_hotels.html')
print("Die Karte wurde unter 'maps/map_hotels.html' gespeichert.")

Die Karte wurde unter 'maps/map_hotels.html' gespeichert.


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [1]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2025-10-06 09:59:09
Python Version: 3.12.1
IP Address: 127.0.0.1
-----------------------------------
